# 1. 处理用户首日通关数目

## 1.1 SQL部分

In [ ]:
-- 1.1 获取2025年7月21号的新增用户,并将其保存到新用户表中
CREATE OR REPLACE TABLE `traffic-jam-master-65a6d.analytics_460528917.new_users20250721` (
    user_pseudo_id string,
    event_date string,
    event_timestamp int64,
    version string
);

INSERT INTO
    `traffic-jam-master-65a6d.analytics_460528917.new_users20250721` (
        user_pseudo_id,
        event_date,
        event_timestamp,
        version
    )
SELECT DISTINCT
    user_pseudo_id,
    event_date,
    event_timestamp,
    app_info.version
FROM
    `traffic-jam-master-65a6d.analytics_460528917.events_intraday_*`
WHERE
    event_name = 'first_open'
    AND _table_suffix='20250721'

In [ ]:
-- 1.2 获取新增用户在7月21号和7月22号两天内的关卡数据，后续用代码处理24小时通关数
CREATE OR REPLACE TABLE `traffic-jam-master-65a6d.analytics_460528917.new_user_20250721_levels_data`(
    user_pseudo_id string,
    event_date string,
    level_id int64,
    level_passed_time int64,
    user_first_open_time int64,
    country string,
    version string
);

INSERT INTO
    `traffic-jam-master-65a6d.analytics_460528917.new_user_20250721_levels_data`(
        user_pseudo_id,
        event_date,
        level_id,
        level_passed_time,
        user_first_open_time,
        country,
        version
    )
SELECT
    DISTINCT
    e.user_pseudo_id,
    e.event_date,
    CAST((SELECT value.string_value FROM UNNEST(e.event_params) WHERE key = 'level_id') AS INT64) AS level_id,
    e.event_timestamp AS level_passed_time,
    nu.event_timestamp AS user_first_open_time,
    e.geo.country,
    e.app_info.version,
FROM
    `traffic-jam-master-65a6d.analytics_460528917.events_intraday_*` e
JOIN
    `traffic-jam-master-65a6d.analytics_460528917.new_users20250721` nu
ON
    nu.user_pseudo_id = e.user_pseudo_id
WHERE
    e.event_name = 'level_up'
    AND e._table_suffix IN ('20250721', '20250722')  -- 获取7月21号和7月22号的数据
    AND e.app_info.version='1.0.9'

## 1.2数据处理部分

In [14]:
# === 版本1.0.9用户专项分析 ===
import pandas as pd
import numpy as np

print("开始分析版本1.0.9的新增用户...")

# 加载数据文件
print("正在读取数据文件...")
new_users = pd.read_csv('csv/NewUser0721.csv')  # 新增用户数据
level_data = pd.read_csv('csv/DayLevelMsg.csv')  # 关卡通过数据

print(f"新增用户数据: {len(new_users)} 条记录")
print(f"关卡数据: {len(level_data)} 条记录")

# 筛选版本1.0.9的新增用户
v109_new_users = new_users[new_users['version'] == '1.0.9'].copy()
print(f"版本1.0.9新增用户数量: {len(v109_new_users)}")

# 获取这些用户的ID
v109_user_ids = set(v109_new_users['user_pseudo_id'])

# 筛选版本1.0.9用户的关卡数据
v109_level_data = level_data[level_data['user_pseudo_id'].isin(v109_user_ids)].copy()
print(f"版本1.0.9用户关卡记录: {len(v109_level_data)} 条")

# 重新计算24小时内通过的关卡
v109_user_first_open = {}
for _, row in v109_new_users.iterrows():
    user_id = row['user_pseudo_id']
    v109_user_first_open[user_id] = row['event_timestamp'] / 1000000  # 转换为秒

# 计算每个用户24小时内通过的关卡
TWENTY_FOUR_HOURS = 86400  # 24小时秒数
v109_user_levels = {}

for user_id in v109_user_ids:
    if user_id in v109_user_first_open:
        first_open_time = v109_user_first_open[user_id]
        
        # 获取该用户的关卡记录
        user_records = v109_level_data[v109_level_data['user_pseudo_id'] == user_id]
        
        # 24小时内通过的关卡
        levels_24h = []
        for _, record in user_records.iterrows():
            level_passed_time = record['level_passed_time'] / 1000000
            time_diff = level_passed_time - first_open_time
            
            if 0 <= time_diff <= TWENTY_FOUR_HOURS:
                levels_24h.append(record['level_id'])
        
        # 修正统计方法：基于最高关卡数而不是去重数量
        # 假设用户是连续通关的，最高关卡数代表实际通关数
        if levels_24h:
            max_level = max(levels_24h)
            # 实际通关数应该是最高关卡数，而不是去重后的关卡列表长度
            v109_user_levels[user_id] = max_level
        else:
            v109_user_levels[user_id] = 0

开始分析版本1.0.9的新增用户...
正在读取数据文件...
新增用户数据: 1154 条记录
关卡数据: 34729 条记录
版本1.0.9新增用户数量: 1152
版本1.0.9用户关卡记录: 34729 条


In [15]:
# === 版本1.0.9用户统计结果 ===
print("\n=== 版本1.0.9新增用户24小时关卡分析结果 ===")

# 1. 每个用户通过的关卡数（修正后基于最高关卡数）
user_level_counts = [levels for levels in v109_user_levels.values()]
print(f"用户通关数列表样例: {user_level_counts[:20]}")  # 显示前20个用户的通关数
active_users = [user for user, levels in v109_user_levels.items() if levels > 0]

print(f"\n📊 基本统计:")
print(f"版本1.0.9新增用户总数: {len(v109_user_ids)}")
print(f"24小时内通过关卡的用户数: {len(active_users)}")
print(f"平均每用户通过关卡数: {sum(user_level_counts)/len(user_level_counts):.1f}")

# 2. 每个用户通过的关卡数（修正输出格式）
print(f"\n🎮 用户通过关卡示例（前10个用户）:")
count = 0
for user_id, max_level in v109_user_levels.items():
    if count >= 10:
        break
    if max_level > 0:
        levels_str = f"1-{max_level}" if max_level > 1 else "1"
    else:
        levels_str = "无"
    print(f"用户 {user_id[:8]}...: {max_level}个关卡 [推测通过: {levels_str}]")
    count += 1

# 3. 保存每个用户的通关情况（修正数据结构）
v109_results = []
for user_id, max_level in v109_user_levels.items():
    levels_passed_str = f"1-{max_level}" if max_level > 1 else ("1" if max_level == 1 else "")
    v109_results.append({
        'user_id': user_id,
        'levels_passed_count': max_level,
        'max_level_reached': max_level,
        'estimated_levels_passed': levels_passed_str
    })

v109_df = pd.DataFrame(v109_results)
v109_df.to_csv('csv/版本1_0_9新增用户24小时关卡_修正版.csv', index=False, encoding='utf-8-sig')
print(f"\n💾 修正后的详细结果已保存到: csv/版本1_0_9新增用户24小时关卡_修正版.csv")
print(f"共 {len(v109_results)} 条用户记录")
print(f"\n🔧 修正说明: 现在基于用户达到的最高关卡数来统计通关数，避免数据丢包导致的低估")
print(f"\n💾 详细结果已保存到: csv/版本1_0_9新增用户24小时关卡.csv")
print(f"共 {len(v109_results)} 条用户记录")


=== 版本1.0.9新增用户24小时关卡分析结果 ===
用户通关数列表样例: [10, 10, 11, 16, 8, 20, 13, 5, 19, 69, 69, 8, 35, 9, 25, 48, 22, 78, 95, 9]

📊 基本统计:
版本1.0.9新增用户总数: 1152
24小时内通过关卡的用户数: 1109
平均每用户通过关卡数: 26.1

🎮 用户通过关卡示例（前10个用户）:
用户 6a4451ff...: 10个关卡 [推测通过: 1-10]
用户 cb74ae2e...: 10个关卡 [推测通过: 1-10]
用户 6ecf2cbe...: 11个关卡 [推测通过: 1-11]
用户 7960029f...: 16个关卡 [推测通过: 1-16]
用户 20795602...: 8个关卡 [推测通过: 1-8]
用户 132743b8...: 20个关卡 [推测通过: 1-20]
用户 c051fbf4...: 13个关卡 [推测通过: 1-13]
用户 3d342c3e...: 5个关卡 [推测通过: 1-5]
用户 f288de4f...: 19个关卡 [推测通过: 1-19]
用户 effdaaa4...: 69个关卡 [推测通过: 1-69]

💾 修正后的详细结果已保存到: csv/版本1_0_9新增用户24小时关卡_修正版.csv
共 1152 条用户记录

🔧 修正说明: 现在基于用户达到的最高关卡数来统计通关数，避免数据丢包导致的低估

💾 详细结果已保存到: csv/版本1_0_9新增用户24小时关卡.csv
共 1152 条用户记录


In [16]:
# === 版本1.0.9用户通关数分布分析 ===
import matplotlib.pyplot as plt
from collections import Counter

print("=== 版本1.0.9用户通关数分布统计 ===")

# 统计不同通关数的用户人数
level_count_distribution = Counter(user_level_counts)

print(f"\n📊 通关数分布统计:")
print(f"{'通关数':<8} {'用户人数':<8} {'占比':<8}")
print("-" * 30)

total_users = len(user_level_counts)
sorted_distribution = sorted(level_count_distribution.items())

for level_count, user_count in sorted_distribution:
    percentage = user_count / total_users * 100
    print(f"{level_count:<8} {user_count:<8} {percentage:.1f}%")

# 保存分布统计到CSV
distribution_data = []
for level_count, user_count in sorted_distribution:
    percentage = user_count / total_users * 100
    distribution_data.append({
        'levels_passed': level_count,
        'user_count': user_count,
        'percentage': round(percentage, 2)
    })
distribution_df = pd.DataFrame(distribution_data)
distribution_df.to_csv('csv/版本1_0_9用户通关数分布.csv', index=False, encoding='utf-8-sig')
print(f"\n💾 分布统计已保存到: csv/版本1_0_9用户通关数分布.csv")

=== 版本1.0.9用户通关数分布统计 ===

📊 通关数分布统计:
通关数      用户人数     占比      
------------------------------
0        43       3.7%
1        13       1.1%
2        14       1.2%
3        27       2.3%
4        20       1.7%
5        22       1.9%
6        35       3.0%
7        24       2.1%
8        98       8.5%
9        32       2.8%
10       68       5.9%
11       24       2.1%
12       62       5.4%
13       35       3.0%
14       52       4.5%
15       29       2.5%
16       104      9.0%
17       12       1.0%
18       34       3.0%
19       20       1.7%
20       47       4.1%
21       6        0.5%
22       18       1.6%
23       8        0.7%
24       27       2.3%
25       27       2.3%
26       8        0.7%
27       13       1.1%
29       9        0.8%
30       10       0.9%
31       3        0.3%
32       4        0.3%
33       3        0.3%
34       3        0.3%
35       18       1.6%
36       1        0.1%
37       1        0.1%
38       4        0.3%
39       4        0.3%
40      

In [17]:
# === 版本1.0.9用户通关数综合分析报告 ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

print("🎯 版本1.0.9新增用户24小时通关数综合分析报告")
print("=" * 60)

# 读取分布数据
distribution_df = pd.read_csv('csv/版本1_0_9用户通关数分布.csv')

# 重构原始数据用于统计计算
levels_data = []
for _, row in distribution_df.iterrows():
    levels_data.extend([int(row['levels_passed'])] * int(row['user_count']))

levels_array = np.array(levels_data)
total_users = len(levels_array)

print(f"\n📊 基础统计信息")
print("-" * 30)
print(f"总用户数: {total_users:,} 人")
print(f"数据时间范围: 2025年7月21日新增用户24小时内通关情况")

# 1. 中心趋势分析
print(f"\n📈 中心趋势分析")
print("-" * 30)

mean_val = np.mean(levels_array)
median_val = np.median(levels_array)
mode_val = Counter(levels_array).most_common(1)[0][0]
mode_count = Counter(levels_array).most_common(1)[0][1]

print(f"算术平均数: {mean_val:.1f} 关卡")
print(f"中位数: {median_val:.1f} 关卡")
print(f"众数: {mode_val} 关卡 (出现 {mode_count} 次)")

# 去除异常值的平均数（去除前后5%）
trimmed_mean = np.mean(np.sort(levels_array)[int(0.05*len(levels_array)):int(0.95*len(levels_array))])
print(f"截尾平均数(去除前后5%): {trimmed_mean:.1f} 关卡")

# 2. 分散程度分析
print(f"\n📊 分散程度分析")
print("-" * 30)

std_val = np.std(levels_array)
var_val = np.var(levels_array)
cv = std_val / mean_val * 100

print(f"标准差: {std_val:.1f}")
print(f"方差: {var_val:.1f}")
print(f"变异系数: {cv:.1f}%")
print(f"极差: {np.max(levels_array) - np.min(levels_array)} (最小: {np.min(levels_array)}, 最大: {np.max(levels_array)})")

# 3. 分位数分析
print(f"\n📋 分位数分析")
print("-" * 30)

percentiles = [10, 25, 50, 75, 90, 95, 99]
for p in percentiles:
    val = np.percentile(levels_array, p)
    print(f"P{p}: {val:.1f} 关卡")

# 4. 用户分层分析
print(f"\n👥 用户分层分析")
print("-" * 30)

# 定义用户分层
def classify_user(level_count):
    if level_count == 0:
        return "非活跃用户"
    elif 1 <= level_count <= 10:
        return "轻度用户"
    elif 11 <= level_count <= 30:
        return "中度用户"
    elif 31 <= level_count <= 50:
        return "重度用户"
    else:
        return "超重度用户"

user_segments = {}
for level_count in levels_array:
    segment = classify_user(level_count)
    user_segments[segment] = user_segments.get(segment, 0) + 1

print(f"{'用户类型':<12} {'人数':<8} {'占比':<8} {'平均通关数':<12}")
print("-" * 45)

segment_order = ["非活跃用户", "轻度用户", "中度用户", "重度用户", "超重度用户"]
for segment in segment_order:
    if segment in user_segments:
        count = user_segments[segment]
        percentage = count / total_users * 100
        
        # 计算该分层的平均通关数
        segment_levels = [level for level in levels_array if classify_user(level) == segment]
        avg_levels = np.mean(segment_levels) if segment_levels else 0
        
        print(f"{segment:<12} {count:<8} {percentage:<7.1f}% {avg_levels:<12.1f}")

# 5. 业务指标分析
print(f"\n💼 关键业务指标")
print("-" * 30)

active_users = sum(1 for level in levels_array if level > 0)
active_rate = active_users / total_users * 100
active_avg = np.mean([level for level in levels_array if level > 0])

high_value_users = sum(1 for level in levels_array if level >= 20)
high_value_rate = high_value_users / total_users * 100

engaged_users = sum(1 for level in levels_array if level >= 10)
engaged_rate = engaged_users / total_users * 100

print(f"用户激活率 (>0关卡): {active_rate:.1f}% ({active_users:,} 人)")
print(f"活跃用户平均通关数: {active_avg:.1f} 关卡")
print(f"深度参与率 (≥10关卡): {engaged_rate:.1f}% ({engaged_users:,} 人)")
print(f"高价值用户率 (≥20关卡): {high_value_rate:.1f}% ({high_value_users:,} 人)")

# 6. 分布特征分析
print(f"\n📊 分布特征分析")
print("-" * 30)

# 偏度和峰度
from scipy import stats
skewness = stats.skew(levels_array)
kurtosis = stats.kurtosis(levels_array)

print(f"偏度: {skewness:.2f} ({'右偏' if skewness > 0 else '左偏' if skewness < 0 else '对称'})")
print(f"峰度: {kurtosis:.2f} ({'尖峰' if kurtosis > 0 else '平峰' if kurtosis < 0 else '正态'})")

# 集中程度分析
top_3_counts = Counter(levels_array).most_common(3)
concentration = sum(count for _, count in top_3_counts) / total_users * 100

print(f"前3个通关数集中度: {concentration:.1f}%")
print(f"最集中的通关数: {[f'{level}关({count}人)' for level, count in top_3_counts]}")

print(f"\n🎯 结论与建议")
print("-" * 30)
print(f"1. 用户参与度很高，{active_rate:.1f}%的用户在24小时内完成了关卡")
print(f"2. 存在明显的用户分化，{high_value_rate:.1f}%的用户展现出高参与度")
print(f"3. 分布呈现右偏特征，少数用户通关数极高(最高{np.max(levels_array)}关)")
print(f"4. 建议关注{trimmed_mean:.0f}关左右的关卡设计，这是大多数用户的核心体验区间")
print(f"5. 可以针对不同用户层级设计差异化的留存策略")

print("\n" + "=" * 60)
print("📄 报告生成完成！")

🎯 版本1.0.9新增用户24小时通关数综合分析报告

📊 基础统计信息
------------------------------
总用户数: 1,152 人
数据时间范围: 2025年7月21日新增用户24小时内通关情况

📈 中心趋势分析
------------------------------
算术平均数: 26.1 关卡
中位数: 15.0 关卡
众数: 16 关卡 (出现 104 次)
截尾平均数(去除前后5%): 19.7 关卡

📊 分散程度分析
------------------------------
标准差: 41.9
方差: 1754.0
变异系数: 160.6%
极差: 530 (最小: 0, 最大: 530)

📋 分位数分析
------------------------------
P10: 4.0 关卡
P25: 8.0 关卡
P50: 15.0 关卡
P75: 24.0 关卡
P90: 57.9 关卡
P95: 95.0 关卡
P99: 210.4 关卡

👥 用户分层分析
------------------------------
用户类型         人数       占比       平均通关数       
---------------------------------------------
非活跃用户        43       3.7    % 0.0         
轻度用户         353      30.6   % 6.9         
中度用户         545      47.3   % 17.6        
重度用户         79       6.9    % 39.8        
超重度用户        132      11.5   % 112.8       

💼 关键业务指标
------------------------------
用户激活率 (>0关卡): 96.3% (1,109 人)
活跃用户平均通关数: 27.1 关卡
深度参与率 (≥10关卡): 71.5% (824 人)
高价值用户率 (≥20关卡): 33.3% (384 人)

📊 分布特征分析
------------------------------
偏度